In [5]:
# Loading libraries and creating common functions

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

from datetime import datetime
from IPython.display import display

theme = 'gridon'

def convertTimeFloat(time):
    # 1666427168
    dateTimeFormat = '%Y-%m-%d %H:%M:%S'
    return pd.to_datetime(
        datetime.fromtimestamp(time, tz=None),
        format = dateTimeFormat
    )

def convertTimeShortString(time):
    # 2022-10-21 13:24:12
    dateTimeFormat = '%Y-%m-%d %H:%M:%S'
    return pd.to_datetime(
        datetime.strptime(time, dateTimeFormat)
    )

def convertTimeLongString(time):
    # 2022-10-21T13:23:56.7964196Z
    dateTimeFormat = '%Y-%m-%dT%H:%M:%S'
    zRemoved = time.split('.')[0]
    return pd.to_datetime(
        datetime.strptime(zRemoved, dateTimeFormat)
    )

def dataFrameFromParserResultType1(folder, file):
    csv = pd.read_csv(folder + file, delimiter=',')
    df = pd.DataFrame()
    df['TimeStamp'] = csv['Time'].apply(convertTimeFloat)
    df['Event'] = csv['EventId']
    df['Parameter0'] = csv['EventId']
    df['Parameter1'] = extractParameters(1, csv['ParameterList'])
    df['Parameter2'] = extractParameters(2, csv['ParameterList'])
    df['Parameter3'] = extractParameters(3, csv['ParameterList'])
    df['Parameter4'] = extractParameters(4, csv['ParameterList'])
    df['Parameter5'] = extractParameters(5, csv['ParameterList'])
    df['Parameter6'] = extractParameters(6, csv['ParameterList'])
    df['Value'] = 1
    df['Row'] = np.arange(len(df))
    return df

def dataFrameFromParserResultType2(folder, file):
    csv = pd.read_csv(folder + file, delimiter=',')
    df = pd.DataFrame()
    df['TimeStamp'] = csv['Time'].apply(convertTimeLongString)
    df['Event'] = csv['EventId']
    df['Parameter0'] = csv['EventId']
    df['Parameter1'] = extractParameters(1, csv['ParameterList'])
    df['Parameter2'] = extractParameters(2, csv['ParameterList'])
    df['Parameter3'] = extractParameters(3, csv['ParameterList'])
    df['Parameter4'] = extractParameters(4, csv['ParameterList'])
    df['Parameter5'] = extractParameters(5, csv['ParameterList'])
    df['Parameter6'] = extractParameters(6, csv['ParameterList'])
    df['Value'] = 1
    df['Row'] = np.arange(len(df))
    return df

def dataFrameFromParserResultType3(folder, file):
    csv = pd.read_csv(folder + file, delimiter=',')
    df = pd.DataFrame()
    df['TimeStamp'] = csv['Time'].apply(convertTimeShortString)
    df['Event'] = csv['EventId']
    df['Parameter0'] = csv['EventId']
    df['Parameter1'] = extractParameters(1, csv['ParameterList'])
    df['Parameter2'] = extractParameters(2, csv['ParameterList'])
    df['Parameter3'] = extractParameters(3, csv['ParameterList'])
    df['Parameter4'] = extractParameters(4, csv['ParameterList'])
    df['Parameter5'] = extractParameters(5, csv['ParameterList'])
    df['Parameter6'] = extractParameters(6, csv['ParameterList'])
    df['Value'] = 1
    df['Row'] = np.arange(len(df))
    return df

def extractParameters(index, column):
    return column.apply(extractParameter, index = index)

def extractParameter(parameter, index):
    temp = parameter.split(",")
    result = list(map(lambda x : x.strip()
        .replace('[','')
        .replace(']','')
        .replace('"','')
        .replace("'","")
        ,temp)
    )
    if index > len(result):
        return ''
    else: 
        return result[index-1]

def mountAggregation(data):
    obj = {}
    eventColumns = list(data)
    for event in eventColumns:
        obj[event] = 'sum'
    obj['TimeStamp'] = 'max'
    return obj

def segmentTimeWindows(dataFrame, segment):
    resultList = []
    last = 0
    counter = 0

    result = dataFrame.resample(segment).groups

    firstTimeStampSample = dataFrame['TimeStamp'][0]
    firstTimeStampSegment = list(result.keys())[0]
    diff = firstTimeStampSample -firstTimeStampSegment
    result = dataFrame.resample(segment, offset = diff).groups

    for key in result:
        data = dataFrame[last:result[key]]
        resultList.append(data)
        last = result[key]
        counter+=1
    return resultList[:-1]

def convertToVector(data):
    events = data.drop('Event', axis=1, level=0).drop('Row', axis=1, level=0).drop('TimeStamp', axis=1, level=0)
    events = events.droplevel(level=0, axis=1)
    events = events.reset_index()
    aggregation = mountAggregation(events)
    aggregated = events.agg(aggregation)
    return aggregated

def transposeData(data, timewindow):
    dataFrameWide = pd.pivot(data, index=['Row','TimeStamp','Event'], columns=['Parameter0'], values=['Value'])
    dataFrameWide = dataFrameWide.reset_index()
    dataFrameWide = dataFrameWide.set_index(pd.DatetimeIndex(dataFrameWide['TimeStamp']))

    segmentList = segmentTimeWindows(dataFrameWide, timewindow)

    vectors = []
    for frame in segmentList:
        vector = convertToVector(frame)
        vectors.append(vector)

    data = pd.concat(vectors, axis=1)
    eventsGrouped = data.transpose()
    return eventsGrouped

def convertToVectorComplete(data):
    events = data.drop('Event', axis=1).drop('Row', axis=1).drop('TimeStamp', axis=1)
    events = events.reset_index()
    aggregation = mountAggregation(events)
    aggregated = events.agg(aggregation)
    return aggregated

def transposeDataComplete(data, timewindow):
    dataFrameWide = pd.pivot(data, index=['Row','TimeStamp','Event'], columns=['Parameter0','Parameter1','Parameter2','Parameter3','Parameter4','Parameter5','Parameter6'], values=['Value'])
    dataFrameWide.columns = dataFrameWide.columns.to_flat_index()
    dataFrameWide = dataFrameWide.reset_index()
    dataFrameWide = dataFrameWide.set_index(pd.DatetimeIndex(dataFrameWide['TimeStamp']))

    segmentList = segmentTimeWindows(dataFrameWide, timewindow)

    vectors = []
    for frame in segmentList:
        vector = convertToVectorComplete(frame)
        vectors.append(vector)

    data = pd.concat(vectors, axis=1)
    eventsGrouped = data.transpose()
    return eventsGrouped

def plotLineEvents(name, data, timewindow):
    eventsGrouped = transposeData(data, timewindow)
    columns = eventsGrouped.columns.drop('TimeStamp')
    fig = px.line(eventsGrouped, x='TimeStamp', y=columns, title=name, template=theme)
    fig.update_traces(patch={"line": {"dash": 'solid'}})
    fig.show()

def plotBoxPlot(name, data, timeWindow):
    eventsGrouped = transposeData(data, timeWindow)
    N = len(eventsGrouped.columns)-1 # Except timestamp column
    c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, N)]

    fig = go.Figure(data = [go.Box(y = eventsGrouped.iloc[:, i + 1],
        marker_color=c[i], name=eventsGrouped.columns[i + 1], boxpoints='suspectedoutliers',)
        for i in range(int(N))]
    )
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        title=name,
        template=theme
    )
    fig.show()

def plotHeapMap(name, data, timeWindow, max = None, height = 800):
    eventsGrouped = transposeData(data, timeWindow)
    events = eventsGrouped.drop(['TimeStamp'], axis=1)
    if(max is None):
        fig = px.imshow(events.T, color_continuous_scale='solar', template=theme)
    else:
        fig = px.imshow(events.T, range_color=[0, max], color_continuous_scale='solar', template=theme)
    fig.update_layout(title=name, height=height)
    fig.show()

def plotHeapMapComplete(name, data, timeWindow, max = None, height = 800, newColumns = None):
    eventsGrouped = transposeDataComplete(data, timeWindow)
    events = eventsGrouped.drop(['TimeStamp'], axis=1)
    
    if(newColumns is not None):
        events.columns = newColumns
    
    if(max is None):
        fig = px.imshow(events.T, color_continuous_scale='solar', template=theme)
    else:
        fig = px.imshow(events.T, range_color=[0, max], color_continuous_scale='solar', template=theme)
    fig.update_layout(title=name, height=height, margin=dict(t=60,r=20,b=40,l=250))
    fig.show()


def reframeColumnNames(name, data, timeWindow):
    eventsGrouped = transposeDataComplete(data, timeWindow)
    events = eventsGrouped.drop(['TimeStamp'], axis=1)
    newColumns = []
    for index, col in enumerate(events.columns):
        text = ','.join(col)
        newColumns.append(text)
    return newColumns


In [4]:
# Loading data

IPLoMFolder = '../Dados/'

IPLoMFile1 = 'abb-edge-17-broker-2.log_structured.csv'
IPLoMFile2 = 'abb-edge-18-broker-1.log_structured.csv'
IPLoMFile3 = 'abb-edge-18-broker-2.log_structured.csv'
IPLoMFile4 = 'abb-edge-19-broker-1.log_structured.csv'
IPLoMFile5 = 'abb-edge-19-broker-2.log_structured.csv'
IPLoMFile6 = 'abb-edge-19-broker-3.log_structured.csv'
IPLoMFile7 = 'abb-edge-19-broker-5.log_structured.csv'
IPLoMFile8 = 'abb-edge-20-broker-3.log_structured.csv'
IPLoMFile9 = 'step2-abb-edge-18-proxy-1.log_structured.csv'
IPLoMFile10 = 'step2-abb-edge-18-proxy-2.log_structured.csv'
IPLoMFile11 = 'step2-abb-edge-19-proxy-1.log_structured.csv'
IPLoMFile12 = 'step2-abb-edge-19-proxy-2.log_structured.csv'
IPLoMFile13 = 'step2-abb-edge-19-proxy-3.log_structured.csv'
IPLoMFile14 = 'step2-abb-edge-19-proxy-5.log_structured.csv'
IPLoMFile15 = 'step1-abb-edge-18-csconnect-1.log_structured.csv'
IPLoMFile16 = 'step1-abb-edge-19-csconnect-1.log_structured.csv'
IPLoMFile17 = 'step1-abb-edge-18-csconnectrouter-1.log_structured.csv'
IPLoMFile18 = 'step1-abb-edge-19-csconnectrouter-1.log_structured.csv'
IPLoMFile19 = 'step1-abb-edge-19-csconnectsimulator-1.log_structured.csv'
IPLoMFile20 = 'step1-abb-edge-kuber-01-csconnect-1.log_structured.csv'

dfList = []
dfList.append({'data': dataFrameFromParserResultType1(IPLoMFolder, IPLoMFile1), 'name': 'abb-edge-17-broker-2'})
dfList.append({'data': dataFrameFromParserResultType1(IPLoMFolder, IPLoMFile2), 'name': 'abb-edge-18-broker-1'})
dfList.append({'data': dataFrameFromParserResultType1(IPLoMFolder, IPLoMFile3), 'name': 'abb-edge-18-broker-2'})
dfList.append({'data': dataFrameFromParserResultType1(IPLoMFolder, IPLoMFile4), 'name': 'abb-edge-19-broker-1'})
dfList.append({'data': dataFrameFromParserResultType1(IPLoMFolder, IPLoMFile5), 'name': 'abb-edge-19-broker-2'})
dfList.append({'data': dataFrameFromParserResultType1(IPLoMFolder, IPLoMFile6), 'name': 'abb-edge-19-broker-3'})
dfList.append({'data': dataFrameFromParserResultType1(IPLoMFolder, IPLoMFile7), 'name': 'abb-edge-19-broker-5'})
dfList.append({'data': dataFrameFromParserResultType1(IPLoMFolder, IPLoMFile8), 'name': 'abb-edge-20-broker-3'})
dfList.append({'data': dataFrameFromParserResultType2(IPLoMFolder, IPLoMFile9), 'name': 'abb-edge-18-proxy-1'})
dfList.append({'data': dataFrameFromParserResultType2(IPLoMFolder, IPLoMFile10), 'name': 'abb-edge-18-proxy-2'})
dfList.append({'data': dataFrameFromParserResultType2(IPLoMFolder, IPLoMFile11), 'name': 'abb-edge-19-proxy-1'})
dfList.append({'data': dataFrameFromParserResultType2(IPLoMFolder, IPLoMFile12), 'name': 'abb-edge-19-proxy-2'})
dfList.append({'data': dataFrameFromParserResultType2(IPLoMFolder, IPLoMFile13), 'name': 'abb-edge-19-proxy-3'})
dfList.append({'data': dataFrameFromParserResultType2(IPLoMFolder, IPLoMFile14), 'name': 'abb-edge-19-proxy-5'})
dfList.append({'data': dataFrameFromParserResultType3(IPLoMFolder, IPLoMFile15), 'name': 'abb-edge-18-csconnect-1'})
dfList.append({'data': dataFrameFromParserResultType3(IPLoMFolder, IPLoMFile16), 'name': 'abb-edge-19-csconnect-1'})
dfList.append({'data': dataFrameFromParserResultType3(IPLoMFolder, IPLoMFile17), 'name': 'abb-edge-18-csconnectrouter-1'})
dfList.append({'data': dataFrameFromParserResultType3(IPLoMFolder, IPLoMFile18), 'name': 'abb-edge-19-csconnectrouter-1'})
dfList.append({'data': dataFrameFromParserResultType3(IPLoMFolder, IPLoMFile19), 'name': 'abb-edge-19-csconnectsimulator-1'})
dfList.append({'data': dataFrameFromParserResultType3(IPLoMFolder, IPLoMFile20), 'name': 'abb-edge-kuber-01-csconnect-1'})



In [ ]:
# Utils

from datetime import datetime, timezone

result1 = datetime.fromtimestamp(1463288494, timezone.utc).isoformat('T', 'microseconds')
print(result1)

result2 = datetime(2022,10,23,7,52,23).strftime('%s')
print(result2)

In [ ]:
dfBroker17_2 = [df for df in dfList if df['name'] == 'abb-edge-17-broker-2'][0]
dfT1 = transposeData(dfBroker17_2['data'], '60T')
df1 = pd.DataFrame(dfT1, columns=['72fc4c37','a32dc633'])

df2 = df1.groupby(['72fc4c37','a32dc633']).size().reset_index(name='count')
display(df2)
fig = px.scatter(df2, x = '72fc4c37', y = 'a32dc633', size='count', color="count")
fig.show()

In [ ]:
dfBroker18_1 = [df for df in dfList if df['name'] == 'abb-edge-18-broker-1'][0]
dfT1 = transposeData(dfBroker18_1['data'], '60T')
df1 = pd.DataFrame(dfT1, columns=['72fc4c37','a32dc633'])

df2 = df1.groupby(['72fc4c37','a32dc633']).size().reset_index(name='count')
display(df2)
fig = px.scatter(df2, x = '72fc4c37', y = 'a32dc633', size='count', color="count")
fig.show()

In [ ]:
dfBroker17_2 = [df for df in dfList if df['name'] == 'abb-edge-17-broker-2'][0]
dfT1 = transposeData(dfBroker17_2['data'], '5T')
df1 = pd.DataFrame(dfT1, columns=['2186d001','a32dc633'])

df2 = df1.groupby(['2186d001','a32dc633']).size().reset_index(name='count')
display(df2)
fig = px.scatter(df2, x = '2186d001', y = 'a32dc633', size='count', color="count")
fig.show()

In [ ]:
dfBroker18_2 = [df for df in dfList if df['name'] == 'abb-edge-18-broker-2'][0]
dfT1 = transposeData(dfBroker18_2['data'], '60T')
df1 = pd.DataFrame(dfT1, columns=['72fc4c37','a32dc633'])

df2 = df1.groupby(['72fc4c37','a32dc633']).size().reset_index(name='count')
display(df2)
fig = px.scatter(df2, x = '72fc4c37', y = 'a32dc633', size='count', color="count")
fig.show()

In [ ]:
xAxis = '54884714'
yAxis = 'a32dc633'
dfBroker19_1 = [df for df in dfList if df['name'] == 'abb-edge-19-broker-1'][0]
dfT1 = transposeData(dfBroker19_1['data'], '60T')
df1 = pd.DataFrame(dfT1, columns=[xAxis, yAxis])

df2 = df1.groupby([xAxis, yAxis]).size().reset_index(name='count')
display(df2)
fig = px.scatter(df2, x = xAxis, y = yAxis, size='count', color="count")
fig.show()

In [ ]:
xAxis = '54884714'
yAxis = 'a32dc633'
dfBroker19_2 = [df for df in dfList if df['name'] == 'abb-edge-19-broker-2'][0]
dfT1 = transposeData(dfBroker19_2['data'], '60T')
df1 = pd.DataFrame(dfT1, columns=[xAxis, yAxis])

df2 = df1.groupby([xAxis, yAxis]).size().reset_index(name='count')
display(df2)
fig = px.scatter(df2, x = xAxis, y = yAxis, size='count', color="count")
fig.show()

In [ ]:
xAxis = '54884714'
yAxis = 'a32dc633'
dfBroker19_3 = [df for df in dfList if df['name'] == 'abb-edge-19-broker-3'][0]
dfT1 = transposeData(dfBroker19_3['data'], '60T')
df1 = pd.DataFrame(dfT1, columns=[xAxis, yAxis])

df2 = df1.groupby([xAxis, yAxis]).size().reset_index(name='count')
display(df2)
fig = px.scatter(df2, x = xAxis, y = yAxis, size='count', color="count")
fig.show()

In [ ]:
xAxis = '72fc4c37'
yAxis = 'a32dc633'
dfBroker19_5 = [df for df in dfList if df['name'] == 'abb-edge-19-broker-5'][0]
dfT1 = transposeData(dfBroker19_5['data'], '60T')
df1 = pd.DataFrame(dfT1, columns=[xAxis, yAxis])

df2 = df1.groupby([xAxis, yAxis]).size().reset_index(name='count')
display(df2)
fig = px.scatter(df2, x = xAxis, y = yAxis, size='count', color="count")
fig.show()

In [7]:
# 1. Broker
timeWindowBroker = '10T'
maxScaleBroker = 80


In [8]:
# 1.1. abb-edge-17-broker-2
dfEdge17Broker2 = [df for df in dfList if df['name'] == 'abb-edge-17-broker-2'][0]
plotLineEvents(dfEdge17Broker2['name'], dfEdge17Broker2['data'], timeWindowBroker)
plotBoxPlot(dfEdge17Broker2['name'], dfEdge17Broker2['data'], timeWindowBroker)
plotHeapMap(dfEdge17Broker2['name'], dfEdge17Broker2['data'], timeWindowBroker, height = 400)

x = reframeColumnNames("Edge17Broker2", dfEdge17Broker2['data'], timeWindowBroker)


In [ ]:
# 1.2. abb-edge-18-broker-1
dfEdge18Broker1 = [df for df in dfList if df['name'] == 'abb-edge-18-broker-1'][0]
plotLineEvents(dfEdge18Broker1['name'], dfEdge18Broker1['data'], timeWindowBroker)
plotBoxPlot(dfEdge18Broker1['name'], dfEdge18Broker1['data'], timeWindowBroker)
plotHeapMap(dfEdge18Broker1['name'], dfEdge18Broker1['data'], timeWindowBroker, height = 350)


In [ ]:
# 1.3. abb-edge-18-broker-2
dfEdge18Broker2 = [df for df in dfList if df['name'] == 'abb-edge-18-broker-2'][0]
plotLineEvents(dfEdge18Broker2['name'], dfEdge18Broker2['data'], timeWindowBroker)
plotBoxPlot(dfEdge18Broker2['name'], dfEdge18Broker2['data'], timeWindowBroker)
plotHeapMap(dfEdge18Broker2['name'], dfEdge18Broker2['data'], timeWindowBroker, height = 350)


In [ ]:
# 1.4. abb-edge-19-broker-1
dfEdge19Broker1 = [df for df in dfList if df['name'] == 'abb-edge-19-broker-1'][0]
plotLineEvents(dfEdge19Broker1['name'], dfEdge19Broker1['data'], timeWindowBroker)
plotBoxPlot(dfEdge19Broker1['name'], dfEdge19Broker1['data'], timeWindowBroker)
plotHeapMap(dfEdge19Broker1['name'], dfEdge19Broker1['data'], timeWindowBroker, height = 350)


In [ ]:
# 1.5. abb-edge-19-broker-2
dfEdge19Broker2 = [df for df in dfList if df['name'] == 'abb-edge-19-broker-2'][0]
plotLineEvents(dfEdge19Broker2['name'], dfEdge19Broker2['data'], timeWindowBroker)
plotBoxPlot(dfEdge19Broker2['name'], dfEdge19Broker2['data'], timeWindowBroker)
plotHeapMap(dfEdge19Broker2['name'], dfEdge19Broker2['data'], timeWindowBroker, height=400)


In [ ]:
# 1.6. abb-edge-19-broker-3
dfEdge19Broker3 = [df for df in dfList if df['name'] == 'abb-edge-19-broker-3'][0]
plotLineEvents(dfEdge19Broker3['name'], dfEdge19Broker3['data'], timeWindowBroker)
plotBoxPlot(dfEdge19Broker3['name'], dfEdge19Broker3['data'], timeWindowBroker)
plotHeapMap(dfEdge19Broker3['name'], dfEdge19Broker3['data'], timeWindowBroker, height=300)


In [ ]:
# 1.7. abb-edge-19-broker-5
dfEdge19Broker5 = [df for df in dfList if df['name'] == 'abb-edge-19-broker-5'][0]
plotLineEvents(dfEdge19Broker5['name'], dfEdge19Broker5['data'], timeWindowBroker)
plotBoxPlot(dfEdge19Broker5['name'], dfEdge19Broker5['data'], timeWindowBroker)
plotHeapMap(dfEdge19Broker5['name'], dfEdge19Broker5['data'], timeWindowBroker, 50, height= 400)



In [ ]:
# 1.8. abb-edge-20-broker-3
dfEdge20Broker3 = [df for df in dfList if df['name'] == 'abb-edge-20-broker-3'][0]
plotLineEvents(dfEdge20Broker3['name'], dfEdge20Broker3['data'], timeWindowBroker)
plotBoxPlot(dfEdge20Broker3['name'], dfEdge20Broker3['data'], timeWindowBroker)
plotHeapMap(dfEdge20Broker3['name'], dfEdge20Broker3['data'], timeWindowBroker, height = 350)

In [ ]:
# Proxy
timeWindowProxy = '30T'
maxScaleProxy = 100


In [ ]:
# 2.1. abb-edge-18-proxy-1
dfEdge18Proxy1 = [df for df in dfList if df['name'] == 'abb-edge-18-proxy-1'][0]
plotLineEvents(dfEdge18Proxy1['name'], dfEdge18Proxy1['data'], timeWindowProxy)
plotBoxPlot(dfEdge18Proxy1['name'], dfEdge18Proxy1['data'], timeWindowProxy)
plotHeapMap(dfEdge18Proxy1['name'], dfEdge18Proxy1['data'], timeWindowProxy)


In [ ]:
# 2.2. abb-edge-18-proxy-2
dfEdge18Proxy2 = [df for df in dfList if df['name'] == 'abb-edge-18-proxy-2'][0]
plotLineEvents(dfEdge18Proxy2['name'], dfEdge18Proxy2['data'], timeWindowProxy)
plotBoxPlot(dfEdge18Proxy2['name'], dfEdge18Proxy2['data'], timeWindowProxy)
plotHeapMap(dfEdge18Proxy2['name'], dfEdge18Proxy2['data'], timeWindowProxy)


In [ ]:
# 2.3. abb-edge-19-proxy-1
time_window_proxy_19_1 = '3T'

dfEdge19Proxy1 = [df for df in dfList if df['name'] == 'abb-edge-19-proxy-1'][0]
plotLineEvents(dfEdge19Proxy1['name'], dfEdge19Proxy1['data'], time_window_proxy_19_1)
plotBoxPlot(dfEdge19Proxy1['name'], dfEdge19Proxy1['data'], time_window_proxy_19_1)
plotHeapMap(dfEdge19Proxy1['name'], dfEdge19Proxy1['data'], time_window_proxy_19_1, height=550)

In [ ]:
# 2.4. abb-edge-19-proxy-2
time_window_proxy_19_2 = '3T'

dfEdge19Proxy2 = [df for df in dfList if df['name'] == 'abb-edge-19-proxy-2'][0]
plotLineEvents(dfEdge19Proxy2['name'], dfEdge19Proxy2['data'], time_window_proxy_19_2)
plotBoxPlot(dfEdge19Proxy2['name'], dfEdge19Proxy2['data'], time_window_proxy_19_2)
plotHeapMap(dfEdge19Proxy2['name'], dfEdge19Proxy2['data'], time_window_proxy_19_2, height=600)

In [ ]:
# 2.5. abb-edge-19-proxy-3
time_window_proxy_19_3 = '3T'

dfEdge19Proxy3 = [df for df in dfList if df['name'] == 'abb-edge-19-proxy-3'][0]
plotLineEvents(dfEdge19Proxy3['name'], dfEdge19Proxy3['data'], time_window_proxy_19_3)
plotBoxPlot(dfEdge19Proxy3['name'], dfEdge19Proxy3['data'], time_window_proxy_19_3)
plotHeapMap(dfEdge19Proxy3['name'], dfEdge19Proxy3['data'], time_window_proxy_19_3)


In [ ]:
# 2.6. abb-edge-19-proxy-5
time_window_proxy_19_5 = '3T'

dfEdge19Proxy5 = [df for df in dfList if df['name'] == 'abb-edge-19-proxy-5'][0]
plotLineEvents(dfEdge19Proxy5['name'], dfEdge19Proxy5['data'], time_window_proxy_19_5)
plotBoxPlot(dfEdge19Proxy5['name'], dfEdge19Proxy5['data'], time_window_proxy_19_5)
plotHeapMap(dfEdge19Proxy5['name'], dfEdge19Proxy5['data'], time_window_proxy_19_5, max = 40, height=850)

In [ ]:
# 3. CSConnect
timeWindowCSConnect = '5T'
maxScaleCSConnect = 100


In [ ]:
# 3.1. abb-edge-18-csconnect-1
dfEdge18CSConnect1 = [df for df in dfList if df['name'] == 'abb-edge-18-csconnect-1'][0]
plotLineEvents(dfEdge18CSConnect1['name'], dfEdge18CSConnect1['data'], timeWindowCSConnect)
plotBoxPlot(dfEdge18CSConnect1['name'], dfEdge18CSConnect1['data'], timeWindowCSConnect)
plotHeapMap(dfEdge18CSConnect1['name'], dfEdge18CSConnect1['data'], timeWindowCSConnect, max=maxScaleCSConnect)


In [ ]:
# 3.2. abb-edge-19-csconnect-1
dfEdgeKuber01CSConnect1 = [df for df in dfList if df['name'] == 'abb-edge-19-csconnect-1'][0]
#plotLineEvents(dfEdge19CSConnect1['name'], dfEdge19CSConnect1['data'], timeWindowCSConnect)
plotBoxPlot(dfEdgeKuber01CSConnect1['name'], dfEdgeKuber01CSConnect1['data'], timeWindowCSConnect)
plotHeapMap(dfEdgeKuber01CSConnect1['name'], dfEdgeKuber01CSConnect1['data'], timeWindowCSConnect, max=50)

In [ ]:
# 3.3. abb-edge-kuber-01-csconnect-1
dfEdgeKuber01CSConnect1 = [df for df in dfList if df['name'] == 'abb-edge-kuber-01-csconnect-1'][0]
plotLineEvents(dfEdgeKuber01CSConnect1['name'], dfEdgeKuber01CSConnect1['data'], timeWindowCSConnect)
plotBoxPlot(dfEdgeKuber01CSConnect1['name'], dfEdgeKuber01CSConnect1['data'], timeWindowCSConnect)
plotHeapMap(dfEdgeKuber01CSConnect1['name'], dfEdgeKuber01CSConnect1['data'], timeWindowCSConnect, max=20, height = 400)

In [ ]:
# 3.3.1 abb-edge-kuber-01-csconnect-1
dfEdgeKuber01CSConnect1 = [df for df in dfList if df['name'] == 'abb-edge-kuber-01-csconnect-1'][0]
dfData = dfEdgeKuber01CSConnect1['data']
dfReceived = dfData[dfData['Event'] == '6ae41321']

newColumns = reframeColumnNames('edgeKuber01CSC1', dfReceived, timeWindowCSConnect)

#plotLineEvents(dfEdgeKuber01CSConnect1['name'], dfReceived, timeWindowCSConnect)
#plotBoxPlot(dfEdgeKuber01CSConnect1['name'], dfReceived, timeWindowCSConnect)
plotHeapMapComplete(dfEdgeKuber01CSConnect1['name'], dfReceived, timeWindowCSConnect, height = 1000, newColumns = newColumns)


In [ ]:
# 4. CSConnect Router
timeWindowCSConnectRouter = '3T'
maxScaleCSConnectRouter = 100


In [ ]:
# 4.1. abb-edge-18-csconnectrouter-1
dfEdge18CSConnectRouter1 = [df for df in dfList if df['name'] == 'abb-edge-18-csconnectrouter-1'][0]
plotLineEvents(dfEdge18CSConnectRouter1['name'], dfEdge18CSConnectRouter1['data'], timeWindowCSConnectRouter)
plotBoxPlot(dfEdge18CSConnectRouter1['name'], dfEdge18CSConnectRouter1['data'], timeWindowCSConnectRouter)
plotHeapMap(dfEdge18CSConnectRouter1['name'], dfEdge18CSConnectRouter1['data'], timeWindowCSConnectRouter, height=400)


In [ ]:
# 4.2. abb-edge-19-csconnectrouter-1
dfEdge19CSConnectRouter1 = [df for df in dfList if df['name'] == 'abb-edge-19-csconnectrouter-1'][0]
plotLineEvents(dfEdge19CSConnectRouter1['name'], dfEdge19CSConnectRouter1['data'], timeWindowCSConnectRouter)
plotBoxPlot(dfEdge19CSConnectRouter1['name'], dfEdge19CSConnectRouter1['data'], timeWindowCSConnectRouter)
plotHeapMap(dfEdge19CSConnectRouter1['name'], dfEdge19CSConnectRouter1['data'], timeWindowCSConnectRouter, height=500)

In [ ]:
# 5. CSConnect Simulator
timeWindowCSConnectSimulator = '10T'
maxScaleCSConnectSimulator = 250


In [ ]:
dfEdge19CSConnectSimulator1 = [df for df in dfList if df['name'] == 'abb-edge-19-csconnectsimulator-1'][0]
plotLineEvents(dfEdge19CSConnectSimulator1['name'], dfEdge19CSConnectSimulator1['data'], timeWindowCSConnectSimulator)
plotBoxPlot(dfEdge19CSConnectSimulator1['name'], dfEdge19CSConnectSimulator1['data'], timeWindowCSConnectSimulator)
plotHeapMap(dfEdge19CSConnectSimulator1['name'], dfEdge19CSConnectSimulator1['data'], timeWindowCSConnectSimulator)
